In [1]:
import folium
import numpy as np
from scipy.spatial.transform import Rotation




In [2]:
def get_circle_coords(lat, lon, radius_miles, num_points=100, reverse=False):

    # Specify circle distance in meters
    dist = 1609.34 * radius_miles

    # Define earth radius in meters
    r_earth = 6.3781e6

    # Define unit vectors
    x_hat = np.array([[1, 0, 0]])
    y_hat = np.array([[0, 1, 0]])
    z_hat = np.array([[0, 0, 1]])



    # Convert to physics theta/phi
    theta = (90 - lat) * np.pi / 180
    phi = lon * np.pi / 180

    # The angle to any point on the circle will be the arc-dist / earth radius
    d_theta = dist / r_earth


    # A unit vector corresponding to the lat/lon
    r1 = np.array(
        [
            np.sin(theta) * np.cos(phi),
            np.sin(theta) * np.sin(phi),
            np.cos(theta),
        ]
    )

    # Find a vector perpendicular to r1 by crossing with south pole
    r_perp = np.cross(r1, -z_hat)

    # Create a rotation by the arc-angle and aligned with the perp vector
    rot_offset = Rotation.from_rotvec(r_perp * d_theta)

    # Apply the rotation to find a single point on the desired circle
    r_offset = rot_offset.apply(r1)

    # Define a series of rotation vectors, all directed along r1
    # but with rotation angles around a circle
    if reverse:
        start, end = 2 * np.pi, 0
    else:
        start, end = 0, 2 * np.pi
        
    alpha = np.linspace(start, end, num_points)
    alpha = np.expand_dims(alpha, -1)
    Alpha = alpha * r1

    # Create a rotation object corresponding to those vectors
    rot_points = Rotation.from_rotvec(Alpha)

    # Create a series of point vectors whos tips all lie on the desired circle
    r_points = rot_points.apply(r_offset)

    # Compute the polar angles for all the circle points
    cos_theta = r_points @ z_hat.T
    theta_points = np.arccos(cos_theta)
    sin_theta = np.sqrt(1 - cos_theta ** 2)

    # Compute the equatorial angle for all reference points
    sin_phi = r_points @ y_hat.T  / sin_theta
    cos_phi = r_points @ x_hat.T / sin_theta
    phi_points = np.arctan2(sin_phi, cos_phi)

    lat_points = 90 - theta_points * 180 / np.pi
    lon_points = 180 * phi_points / np.pi

    # Make coords in the format folium likes
    coords = [list(t) for t in zip(lat_points.flatten(), lon_points.flatten())]
    
    return coords


In [5]:
def add_ring(m, lat, lon):

    coords1 = get_circle_coords(lat, lon, radius_miles=550, num_points=100, reverse=True)
    coords2 = get_circle_coords(lat, lon, radius_miles=1220, num_points=100)
    coords = coords2 + coords1


    folium.vector_layers.Polygon(
        coords,
        color='blue',
        fill=True,
        weight=0,
        fill_opacity=.1
    ).add_to(m)
    return m

chattanooga = [35.054877, -85.067434]
tulsa = [36.081775, -95.931575]
show_low = [34.256688, -110.021647]
seattle = [47.542368, -122.355455]
boston = [42.283284, -71.143324]
chicago = [41.770176, -87.900197]
pasadena = [34.156212, -118.146551]

locs = [
    chattanooga,
    tulsa,
    show_low,
    seattle,
    boston,
    chicago,
    pasadena
]

chattanooga = [35.054877, -85.067434]
m = folium.Map(location=chattanooga, zoom_start=4)
for loc in locs:
    m = add_ring(m, *loc)
    folium.Marker(loc).add_to(m)
m
